In [1]:
import os
import subprocess
from pathlib import Path
import numpy as np
import operator
import json
import math
import resource

In [2]:
DIVISIONS = ['div1', 'div2', 'all']

DIV_THRESHOLD = 1500

In [3]:
division = 'all'

In [4]:
time_limit = 10
memory_limit = 256 * 1024 * 1024

In [5]:
all_students = ['94101737', '94101975', '95105657', '95106494', '95108008', '95109242', '95109383', '96101262', '96101913', '96104881', '96105553', '96105575', '96105626', '96105637', '96105672', '96105683', '96105712', '96105807', '96105829', '96105853', '96105864', '96105875', '96105886', '96105948', '96105983', '96106033', '96106077', '96106117', '96109588', '96109599', '96109606', '96109663', '96109714', '96109736', '96109747', '96109758', '96109899', '96109906', '96110067', '96110129', '96110797', '97100581', '97102044', '97102441', '97105696', '97105771', '97105844', '97105855', '97106262', '97106295', '97106419', '97110155', '97110206', '97213093', '98209048']

In [6]:
import shutil

working_directory = os.path.abspath('')
os.chdir(working_directory)

submissions_path = Path(os.path.join(working_directory, 'submissions'))

tests_path = Path(os.path.join(working_directory, 'tests'))

executables_path = os.path.join(working_directory, 'executables')

rating_file = 'rating.json'

if os.path.exists(executables_path):
    shutil.rmtree(executables_path)

os.mkdir(executables_path)
    
outputs_path = Path(os.path.join(working_directory, 'outputs'))
    
if os.path.exists(outputs_path):
    shutil.rmtree(outputs_path)    
    
os.mkdir(outputs_path)

In [7]:
students = list(os.walk(submissions_path))[0][1]

if division != 'div1':
    students += ['dummy']

try:
    with open(rating_file, 'r', encoding='utf-8') as fp:
        rate = json.load(fp)

    for student in students:
        if student not in rate:
            print(f'FATAL ERROR: {student} has no rating!')

    for std in rate:
        if std not in students:
            print(f'{std} did not submit at this round.')
            students.append(std)
except FileNotFoundError:
    print('No rating file.')

if division == 'div1':
    students = list(filter(lambda x: rate[x] >= DIV_THRESHOLD, students))
elif division == 'div2':
    students = list(filter(lambda x: rate[x] < DIV_THRESHOLD, students))
            
final_results = {student: {'ranking': None, 'tests': {}, 'final_score': 0, 'prev_rating': 0, 'new_rating': 0} for student in students}

No rating file.


NameError: name 'rate' is not defined

In [12]:
print(division, 'contest')
print(f'{len(students)} students we have: {students}')

all contest
40 students we have: ['97100581', '97102044', '97106419', '96109599', '94101737', '96105875', '97105771', '96105886', '96110797', '96105829', '96109606', '96109663', '96110129', '97106262', '96105712', '97105844', '96105948', '96105983', '96109736', '96105553', '97102441', '94170733', '96109906', '97105855', '97105696', '98209048', '96105683', '97110206', '95109383', '95108008', '96101262', '96106077', '96105807', '96110067', '97106295', '96109588', '97110155', '96109747', '96109714', 'dummy']


In [13]:
print(submissions_path)
print(working_directory)
print(tests_path)
print(executables_path)

/Users/kianooshabbasi/Dropbox/DA99/Covid Challenge/Round-1/judge/submissions
/Users/kianooshabbasi/Dropbox/DA99/Covid Challenge/Round-1/judge
/Users/kianooshabbasi/Dropbox/DA99/Covid Challenge/Round-1/judge/tests
/Users/kianooshabbasi/Dropbox/DA99/Covid Challenge/Round-1/judge/executables


In [14]:
all_tests = list(map(lambda path: os.path.basename(path).split('in')[1].split('.txt')[0], tests_path.rglob('*.txt')))

In [18]:
print('Compiling codes...')

errors = 0

for student in students:
    cpp_files = list(Path(os.path.join(submissions_path, student)).rglob('*.c*'))
    ans = [] 
    for i in cpp_files:
        if i.name.endswith(".cxx") or i.name.endswith(".cpp"):
            ans.append(i)
    cpp_files = ans
    
    if len(cpp_files) != 1:
        print(f'Something wrong with {student}\'s files. Could not find the cpp file')
        final_results[student]['tests'] = {test: {'score': 0, 'cost': None, 'status': 'No Submission'} for test in all_tests}
        errors += 1
        continue

    cpp_file = cpp_files[0]
    output_file = os.path.join(submissions_path, student, 'output-runnable')
    
    process = subprocess.run(['g++', cpp_file, '-w', '-DONLINE_JUDGE', 
                              '-std=c++14', '-fno-asm', '-Dasm=error', '-lm',
                              '-O2', '-o', output_file], capture_output=True)
    if process.returncode:
        print(f'Could not compile {student}\'s code.')
        print(f'*******************')
        print(process.stderr.decode()[:300])
        print(f'*******************')              
        final_results[student]['tests'] = {test: {'score': 0, 'cost': None, 'status': 'Compiler Error'} for test in all_tests}
        errors += 1
        continue
    
    
    shutil.move(output_file, os.path.join(executables_path, f'{student}.exec'))

print(f'Compilation finished for {len(students)} codes: {errors} error(s).')
print('\n=================================================\n')

Compiling codes...
Something wrong with dummy's files. Could not find the cpp file
Compilation finished for 40 codes: 1 error(s).




In [19]:
print('Executing codes...')

def limit_virtual_memory():
    resource.setrlimit(resource.RLIMIT_AS, (memory_limit, memory_limit))


for executable in list(Path(executables_path).rglob('*.exec')):    
    student_id = os.path.basename(executable).split('.exec')[0]
        
    print(f'Running {student_id}\'s codes')        
        
    for test_file in tests_path.rglob('*.txt'):
        test_name = os.path.basename(test_file).split('.txt')[0]
        
        test_num = test_name[2:]
        
        output_dir = os.path.join(outputs_path, test_name)
        os.makedirs(output_dir, exist_ok=True)
        
        output_path = os.path.join(output_dir, f'output-{student_id}.txt')
        
        input_file = open(test_file)
        output_file = open(output_path, 'w')
        
        try:
            subprocess.run([executable], stdin=input_file, stdout=output_file, timeout=time_limit, preexec_fn=limit_virtual_memory, check=True)
            output_file.close()
        except subprocess.TimeoutExpired:
            print(f'Time limit exceeded on test {test_name}')
            final_results[student_id]['tests'][test_num] = {'score': 0, 'cost': None, 'status': 'Time limit'}
            os.remove(output_path)
        except subprocess.CalledProcessError:
            print(f'Runtime error on test {test_name}')
            final_results[student_id]['tests'][test_num] = {'score': 0, 'cost': None, 'status': 'Runtime Error'}
            os.remove(output_path)
        
        input_file.close()
        
print('\n=================================================\n')

Executing codes...
Running 97105696's codes
Running 97110206's codes
Running 96109599's codes
Running 96109736's codes
Running 97106262's codes
Running 97105771's codes
Running 96101262's codes
Running 97110155's codes
Running 96105807's codes
Running 97105855's codes
Time limit exceeded on test input20
Time limit exceeded on test input15
Time limit exceeded on test input14
Running 96109747's codes
Running 97106419's codes
Running 98209048's codes
Running 96105683's codes
Runtime error on test input17
Running 96105553's codes
Running 97105844's codes
Running 94101737's codes
Time limit exceeded on test input19
Time limit exceeded on test input18
Time limit exceeded on test input3
Time limit exceeded on test input7
Time limit exceeded on test input6
Time limit exceeded on test input20
Time limit exceeded on test input5
Time limit exceeded on test input8
Time limit exceeded on test input10
Time limit exceeded on test input11
Runtime error on test input9
Time limit exceeded on test input1

In [9]:
checker_files = list(Path(working_directory).rglob('checker.cpp')) 
if len(checker_files) != 1:
    print("Please provide one checker.cpp file")

judge = os.path.join(working_directory, 'judge.exec')    

process = subprocess.run(['g++', checker_files[0], '-std=c++14', '-O2', '-o', judge], capture_output=True)
if process.returncode:
    print(f'Could not compile {student}\'s code:')
    print(process.stderr.decode()[:300])

In [21]:
cost = {}
for s in students:
    cost[s] = {}

for test in list(Path(outputs_path).rglob('in*')):
    this_test = os.path.basename(test) 
    test_num = this_test[2:]
    
    input_file = os.path.join(tests_path, this_test + '.txt')
    
    valid_submissions = list(test.rglob('output*.txt'))
    print("Seeing submissions output for test ", test_num, "\n")

    for student_output in valid_submissions:
        student_id = student_output.name.split('output-')[1].split('.txt')[0]
        
        try:
            out_path = os.path.join(test, f'checker-{student_id}.txt')
            log_path = os.path.join(test, f'log-{student_id}.txt')
            out_file = open(out_path, 'w')
            log_file = open(log_path, 'w')
            
            subprocess.run([judge, input_file, student_output], timeout=2, stdout=out_file, stderr=log_file, check=True)
            
            with open(out_path, "r") as f:
                checker_out = f.read()
            
            out_file.close()
            log_file.close()
            
            cost[student_id][test_num] = int(checker_out)
            print(f"{student_id}'s cost is {checker_out}")
        except subprocess.TimeoutExpired:
            print(f'Time limit exceeded on checker. its not normal')
            os.remove(log_file)
        except subprocess.CalledProcessError:
            print(f'Runtime error on checker. its not normal')
            os.remove(log_file)
        
        

Seeing submissions output for test  put16 

96109736's cost is 27377347445

96105683's cost is 14706630352

96110067's cost is 9964867226

97106262's cost is 10963997389

97110206's cost is 11752724826

96109906's cost is -1

96105875's cost is 11944867034

95108008's cost is 10963997389

96105807's cost is 19713650579

96106077's cost is 11298474936

97105696's cost is 11798950837

96105829's cost is 10081550648

96105948's cost is 23714388558

96109747's cost is 15257624399

96105553's cost is 16690363065

96110797's cost is 16690363065

96109588's cost is 2136354063596

97106419's cost is 19149120776

97102441's cost is 14436807728

97105844's cost is 10213472960

98209048's cost is 12900870450

96101262's cost is 21224404412

96109606's cost is 11766719899

96109599's cost is -1

95109383's cost is -1

97105855's cost is 10028010874

96105886's cost is 16690363065

96109663's cost is 10587661565

94170733's cost is 10554162157

97106295's cost is -1

96109714's cost is -1

97110155

96105683's cost is 11965566492

96110067's cost is 6952592999

97106262's cost is 11406609188

97110206's cost is 10737796328

96109906's cost is -1

96105875's cost is 12363669109

95108008's cost is 9221779084

96105807's cost is 15217312093

96106077's cost is 7595906305

97105696's cost is 8045008030

96105829's cost is 7595906284

96105948's cost is 13153839346

96109747's cost is 10569084350

96105553's cost is 12948125987

96110797's cost is 12948125987

96109588's cost is 19304101370368

97106419's cost is 12948125987

97102441's cost is 8045008032

97105844's cost is 7036949002

97102044's cost is 10229838136

98209048's cost is 11355610120

96101262's cost is 15217312093

96109606's cost is 8966888848

96109599's cost is -1

95109383's cost is -1

97105855's cost is 6952593001

96105886's cost is 12948125987

96109663's cost is 8688321336

94170733's cost is 8045008030

97106295's cost is -1

96109714's cost is -1

94101737's cost is -1

97110155's cost is 21771802405

971005

97106419's cost is 15670684378

97102441's cost is 13605499463

97105844's cost is 2915996738

97102044's cost is 2915935952

98209048's cost is 7370172268

96101262's cost is 17534535341

96109606's cost is 7774796702

96109599's cost is -1

95109383's cost is -1

97105855's cost is 2916357375

96105886's cost is 11176590366

96109663's cost is 4859481305

94170733's cost is 6803247568

97106295's cost is -1

96109714's cost is -1

97110155's cost is 19032582120

97100581's cost is 5347024895

96105712's cost is -1

97105771's cost is 8663150194

Seeing submissions output for test  put8 

96109736's cost is 551060

96105683's cost is 641616

96110067's cost is 373943

97106262's cost is 551060

97110206's cost is 523516

96109906's cost is -1

96105875's cost is 464461

95108008's cost is 492022

96105807's cost is 150542682

96106077's cost is 373970

97105696's cost is 547092

96105829's cost is 75180863

96105948's cost is 599116400

96109747's cost is 751758

96105553's cost is 66

In [26]:
def prob(ri, rj):
    delta = (rj - ri) / 400.
    return 1.0 / (1.0 + 10**(delta))

def return_student_ranking(all_scores, all_tests, test_weights, use_w = False):
    total_score = {student: 0 for student in students}
    for t in all_tests:
        results = []
        for s in students:
            if not t in all_scores[s]:
                continue
            res = all_scores[s][t]
            if res != -1:
                results.append((res, s))
        results.sort()
        first_three = results
        if len(results) > 3:
            first_three = first_three[:3]
        C = []
        for i in range(len(first_three)):
            C.append(first_three[i][0])
        Cm = np.mean(C)
        for s in students:
            if not t in all_scores[s]:
                continue
            res = all_scores[s][t]
            if res == -1:
                final_results[s]['tests'][t] = {'score': 0, 'cost': None, 'status': 'Invalid'}
                continue
            X = test_weights[t] * min(1, (Cm/res)**(1/3))
            final_results[s]['tests'][t] = {'score': round(X, 3), 'cost': res, 'status': 'Success'}
            total_score[s] += X
    sorted_d = sorted(total_score.items(), key=operator.itemgetter(1), reverse=True)
    rank = []
    eps = 1e-6
    for i, (student, score) in enumerate(sorted_d):
        cnt = 1
        for i2, (student2, score2) in enumerate(sorted_d):
            if score2 - eps > score:
                cnt += 1
        pair_rank = (student, cnt)
        rank.append(pair_rank)
        
        final_results[student]['final_score'] = round(score, 3)
        final_results[student]['ranking'] = cnt
    return rank

def get_new_ratings(rate, standings): # current rates, standing of student ids in latest round
    seed = {}
    for i in rate:
        seed[i] = 1
        for j in rate:
            if i == j:
                continue
            seed[i] += prob(rate[j], rate[i])
    new_rates = {}
    for idx, rank in enumerate(standings): # u: user
        lo, hi = 10, 4000
        u = rank[0]
        place = rank[1]
        m_i = math.sqrt((place) * seed[u])
        while(hi - lo > 1):
            mid = (hi + lo)//2
            new_seed = 1
            for j in rate:
                if j == u:
                    continue
                new_seed += prob(rate[j], mid)
            
            if new_seed < m_i: # over valued rating
                hi = mid
            else:
                lo = mid
        R = (hi + lo) * 0.5
        d_i = (R - rate[u]) * 0.5
        new_rates[u] = int(rate[u] + d_i)
    return new_rates


def update_ratings():
    rating_files = list(Path(working_directory).rglob(rating_file))
    rate = {}
    if len(rating_files) == 0:
        print("NO ratings will set every one to 1500")
        for s in students:
            rate[s] = 1500
    else:
        with open(rating_file, 'r', encoding='utf-8') as fp:
            rate = json.load(fp)
    print("previous rates -----")
    print(rate)
    for x, y in rate.items():
        final_results[x]['prev_rating'] = y
    standings = return_student_ranking(cost, all_tests, {t: 100 for t in all_tests}, False)
    new_rates = get_new_ratings(rate, standings)
    destination = f"rating-{division}.json"
    print("new rates -----")
    print(new_rates)
    for x, y in new_rates.items():
        final_results[x]['new_rating'] = y
    with open(destination, 'w', encoding='utf-8') as fp:
        json.dump(new_rates, fp)

In [27]:
update_ratings()

NO ratings will set every one to 1500
previous rates -----
{'97100581': 1500, '97102044': 1500, '97106419': 1500, '96109599': 1500, '94101737': 1500, '96105875': 1500, '97105771': 1500, '96105886': 1500, '96110797': 1500, '96105829': 1500, '96109606': 1500, '96109663': 1500, '96110129': 1500, '97106262': 1500, '96105712': 1500, '97105844': 1500, '96105948': 1500, '96105983': 1500, '96109736': 1500, '96105553': 1500, '97102441': 1500, '94170733': 1500, '96109906': 1500, '97105855': 1500, '97105696': 1500, '98209048': 1500, '96105683': 1500, '97110206': 1500, '95109383': 1500, '95108008': 1500, '96101262': 1500, '96106077': 1500, '96105807': 1500, '96110067': 1500, '97106295': 1500, '96109588': 1500, '97110155': 1500, '96109747': 1500, '96109714': 1500, 'dummy': 1500}
new rates -----
{'97100581': 1700, '96109663': 1658, '96106077': 1634, '96109606': 1616, '97110206': 1603, '97105844': 1591, '95108008': 1581, '97105696': 1572, '96105712': 1564, '97102441': 1557, '97105855': 1550, '9610582

/Users/kianooshabbasi/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars


In [28]:
data = sorted(final_results.items(), key= lambda item: item[1]['ranking'])

In [29]:
data

[('97100581',
  {'ranking': 1,
   'tests': {'put1': {'score': 100, 'cost': 5000002, 'status': 'Success'},
    'put19': {'score': 98.361, 'cost': 34471165546, 'status': 'Success'},
    'put18': {'score': 90.627, 'cost': 46042857060, 'status': 'Success'},
    'put2': {'score': 100, 'cost': 2000300020, 'status': 'Success'},
    'put3': {'score': 95.655, 'cost': 10475876, 'status': 'Success'},
    'put7': {'score': 100.0, 'cost': 10564135670, 'status': 'Success'},
    'put6': {'score': 97.646, 'cost': 15310216077, 'status': 'Success'},
    'put4': {'score': 100, 'cost': 6952592999, 'status': 'Success'},
    'put20': {'score': 100, 'cost': 0, 'status': 'Success'},
    'put5': {'score': 100, 'cost': 1142722486, 'status': 'Success'},
    'put8': {'score': 99.997, 'cost': 373988, 'status': 'Success'},
    'put10': {'score': 100, 'cost': 8126830437, 'status': 'Success'},
    'put11': {'score': 99.66, 'cost': 16344138532, 'status': 'Success'},
    'put9': {'score': 81.701, 'cost': 5347024895, 's

In [30]:
import xlsxwriter


def export_excel(data):

    workbook = xlsxwriter.Workbook(f'results-{division}.xlsx')
    worksheet = workbook.add_worksheet()
    cell_format1 = workbook.add_format({'bg_color': 'green'})
    cell_format2 = workbook.add_format({'bg_color': 'red'})

    col = 0
    worksheet.write(0, 0, "Ranking")
    worksheet.write(0, 1, "Student ID")
    worksheet.write(0, 2, "Final score")
    worksheet.write(0, 3, "Prev rating")
    worksheet.write(0, 4, "Rating change")
    worksheet.write(0, 5, "New rating")
    worksheet.write(0, 6, "")

    for i in range(1,21):
        worksheet.write(0, i + 6, f"Test {i} score\n(cost)")
        
    for i in range(len(data)):
        col = 0
        worksheet.write(2*i+1, col, data[i][1]["ranking"])
        worksheet.write(2*i+2, col, "")
        col += 1
        worksheet.write(2*i+1, col, data[i][0])
        worksheet.write(2*i+2, col, "")
        col += 1
        worksheet.write(2*i+1, col, data[i][1]["final_score"])
        worksheet.write(2*i+2, col, "")
        col += 1
        worksheet.write(2*i+1, col, data[i][1]["prev_rating"])
        worksheet.write(2*i+2, col, "")
        col += 1
        worksheet.write(2*i+1, col, data[i][1]["new_rating"] - data[i][1]["prev_rating"])
        worksheet.write(2*i+2, col, "")
        col += 1
        worksheet.write(2*i+1, col, data[i][1]["new_rating"])
        worksheet.write(2*i+2, col, "")
        col += 1
        worksheet.write(2*i+1, col, "")
        worksheet.write(2*i+2, col, "")
        col += 1
        for num in sorted(map(lambda x: x[3:], data[i][1]["tests"]), key=int):
            worksheet.write(2 * i + 1, col, data[i][1]["tests"][f"put{num}"]["score"])
            if data[i][1]["tests"][f"put{num}"]['status'] == "Success":
                worksheet.write(2 * i + 2, col, f'({data[i][1]["tests"][f"put{num}"]["cost"]})')
            else:
                worksheet.write(2 * i + 2, col, f'({data[i][1]["tests"][f"put{num}"]["status"]})')
        
            col += 1

    workbook.close()

In [31]:
export_excel(data)

In [10]:
os.remove(judge)